In [1]:
import pandas as pd
import geopandas as gpd
import requests, json, geojson
from shapely.geometry import shape
import shapely.wkt
import os

import sys
sys.path.append('../scripts/')
from utility import read_file, create_folder, temp_record_query, temp_record_sdf

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
from collections import defaultdict

### import data

- POA(shapefile)

In [3]:
poa_sf = gpd.read_file('../data/tables/external_POA/POA_2021_AUST_GDA2020.shp')
poa_sf['geometry'] = poa_sf['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
poa_gdf = gpd.GeoDataFrame(poa_sf)

In [4]:
poa_gdf.head()

,POA_CODE21,POA_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,SHAPE_Leng,SHAPE_Area,geometry
0,0800,0800,AUS,Australia,3.1731,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.081893,0.000264,"POLYGON ((130.83681 -12.45376, 130.83684 -12.4..."
1,0810,0810,AUS,Australia,24.4283,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.241859,0.002031,"POLYGON ((130.89986 -12.36567, 130.89875 -12.3..."
2,0812,0812,AUS,Australia,35.8899,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.278788,0.002983,"POLYGON ((130.91915 -12.40786, 130.91831 -12.4..."
3,0820,0820,AUS,Australia,39.0642,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.409134,0.003248,"POLYGON ((130.85260 -12.43994, 130.85089 -12.4..."
4,0822,0822,AUS,Australia,150775.8030,http://linked.data.gov.au/dataset/asgsed3/POA/...,90.601831,12.564238,"MULTIPOLYGON (((136.56648 -12.08392, 136.56704..."


In [5]:
poa_gdf.count()

POA_CODE21    2644
POA_NAME21    2644
AUS_CODE21    2644
AUS_NAME21    2644
AREASQKM21    2644
LOCI_URI21    2644
SHAPE_Leng    2644
SHAPE_Area    2644
geometry      2641
dtype: int64

In [6]:
poa_gdf[poa_gdf.geometry.isna()]

,POA_CODE21,POA_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,SHAPE_Leng,SHAPE_Area,geometry
2641,9494,No usual address (Aust.),AUS,Australia,0.0,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.0,0.0,None
2642,9797,Migratory - Offshore - Shipping (Aust.),AUS,Australia,0.0,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.0,0.0,None
2643,ZZZZ,Outside Australia,ZZZ,Outside Australia,0.0,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.0,0.0,None


In [7]:
poa_gdf = poa_gdf.replace(to_replace='None', value=0).dropna()

In [8]:
poa_gdf = poa_gdf[['POA_CODE21', 'geometry']]

- SA2(shapefile)

In [9]:
sa2_sf = gpd.read_file('../data/tables/external_SA2/SA2_2021_AUST_GDA2020.shp')
sa2_sf['geometry'] = sa2_sf['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
sa2_sf = gpd.GeoDataFrame(sa2_sf)

In [10]:
sa2_sf.head()

,SA2_CODE21,SA2_NAME21,CHG_FLAG21,CHG_LBL21,SA3_CODE21,SA3_NAME21,SA4_CODE21,SA4_NAME21,GCC_CODE21,GCC_NAME21,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,geometry
0,101021007,Braidwood,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,3418.3525,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((149.58424 -35.44426, 149.58444 -35.4..."
1,101021008,Karabar,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,6.9825,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((149.21899 -35.36738, 149.21800 -35.3..."
2,101021009,Queanbeyan,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,4.7620,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((149.21326 -35.34325, 149.21619 -35.3..."
3,101021010,Queanbeyan - East,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,13.0032,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((149.24034 -35.34781, 149.24024 -35.3..."
4,101021012,Queanbeyan West - Jerrabomberra,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,13.6748,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((149.19572 -35.36126, 149.19970 -35.3..."


In [11]:
sa2_sf = sa2_sf[['SA2_CODE21', 'geometry']]

- SA2 population data

In [12]:
sa2_data = pd.read_csv('../data/tables/external_SA2_data.csv')

In [13]:
sa2_dt_head = sa2_data.loc[:4]
sa2_dt = sa2_data.loc[5:]

In [14]:
sa2_dt.head()

,Unnamed: 0,Australian Bureau of Statistics,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,...,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36
5,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2018,2019,2020,2021,NaN,2011-2021,NaN,NaN,Area,Population density 2021
6,6,S/T code,S/T name,GCCSA code,GCCSA name,SA4 code,SA4 name,SA3 code,SA3 name,SA2 code,...,no.,no.,no.,no.,NaN,no.,%,NaN,km2,persons/km2
7,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,1,New South Wales,1RNSW,Rest of NSW,101,Capital Region,10102,Queanbeyan,101021007,...,4140,4211,4273,4330,NaN,872,25.2,NaN,3418.4,1.3
9,9,1,New South Wales,1RNSW,Rest of NSW,101,Capital Region,10102,Queanbeyan,101021008,...,8507,8488,8519,8546,NaN,-546,-6,NaN,7,1223.9


In [15]:
sa2_dt = sa2_dt[list(sa2_dt.columns)[9:10] + list(sa2_dt.columns)[30:32]+list(sa2_dt.columns)[36:37]]
sa2_dt.head()

,Unnamed: 8,Unnamed: 29,Unnamed: 30,Unnamed: 35
5,NaN,2020,2021,Area
6,SA2 code,no.,no.,km2
7,NaN,NaN,NaN,NaN
8,101021007,4273,4330,3418.4
9,101021008,8519,8546,7


In [16]:
sa2_dt.count()

Unnamed: 8     2455
Unnamed: 29    2457
Unnamed: 30    2457
Unnamed: 35    2457
dtype: int64

In [17]:
col_name1 = sa2_dt.iloc[:1].to_numpy().tolist()[0]
col_name2 = sa2_dt.iloc[1:2].to_numpy().tolist()[0]
origin_col = list(sa2_dt.columns)

In [18]:
col_dict = defaultdict()
for i in range(4):
    if i == 0:
        col_dict[origin_col[i]] = col_name2[i]
    else:
        col_dict[origin_col[i]] = col_name1[i] + '(' + str(col_name2[i]) + ')'

In [19]:
col_dict

defaultdict(None,
            {'Unnamed: 8': 'SA2 code',
             'Unnamed: 29': '2020(no.)',
             'Unnamed: 30': '2021(no.)',
             'Unnamed: 35': 'Area(km2)'})

In [20]:
sa2_dt = sa2_dt.iloc[3:2457].reset_index(drop=True).rename(col_dict, axis=1)

In [21]:
sa2_dt.head()

,SA2 code,2020(no.),2021(no.),Area(km2)
0,101021007,4273,4330,3418.4
1,101021008,8519,8546,7
2,101021009,11437,11370,4.8
3,101021010,5077,5093,13
4,101021012,12796,12743,13.7


### SA2 geo join with SA2 data

In [22]:
sa2_sf.head()

,SA2_CODE21,geometry
0,101021007,"POLYGON ((149.58424 -35.44426, 149.58444 -35.4..."
1,101021008,"POLYGON ((149.21899 -35.36738, 149.21800 -35.3..."
2,101021009,"POLYGON ((149.21326 -35.34325, 149.21619 -35.3..."
3,101021010,"POLYGON ((149.24034 -35.34781, 149.24024 -35.3..."
4,101021012,"POLYGON ((149.19572 -35.36126, 149.19970 -35.3..."


In [23]:
sa2_dt.count()

SA2 code     2454
2020(no.)    2454
2021(no.)    2454
Area(km2)    2454
dtype: int64

In [24]:
sa2_sf.count()

SA2_CODE21    2473
geometry      2454
dtype: int64

In [25]:
sa2_sf = sa2_sf.dropna()
sa2_dt = sa2_dt.reset_index(drop=True)
sa2_sf.count()

SA2_CODE21    2454
geometry      2454
dtype: int64

In [26]:
sa2 = pd.merge( sa2_sf, sa2_dt, left_on='SA2_CODE21', right_on='SA2 code', how='inner').drop('SA2 code', axis=1)

In [27]:
sa2.head()

,SA2_CODE21,geometry,2020(no.),2021(no.),Area(km2)
0,101021007,"POLYGON ((149.58424 -35.44426, 149.58444 -35.4...",4273,4330,3418.4
1,101021008,"POLYGON ((149.21899 -35.36738, 149.21800 -35.3...",8519,8546,7
2,101021009,"POLYGON ((149.21326 -35.34325, 149.21619 -35.3...",11437,11370,4.8
3,101021010,"POLYGON ((149.24034 -35.34781, 149.24024 -35.3...",5077,5093,13
4,101021012,"POLYGON ((149.19572 -35.36126, 149.19970 -35.3...",12796,12743,13.7


### geo intersection

In [28]:
sa2['2020(no.)'] = sa2['2020(no.)'].astype('int')
sa2['2021(no.)'] = sa2['2021(no.)'].astype('int')
sa2.head()

,SA2_CODE21,geometry,2020(no.),2021(no.),Area(km2)
0,101021007,"POLYGON ((149.58424 -35.44426, 149.58444 -35.4...",4273,4330,3418.4
1,101021008,"POLYGON ((149.21899 -35.36738, 149.21800 -35.3...",8519,8546,7
2,101021009,"POLYGON ((149.21326 -35.34325, 149.21619 -35.3...",11437,11370,4.8
3,101021010,"POLYGON ((149.24034 -35.34781, 149.24024 -35.3...",5077,5093,13
4,101021012,"POLYGON ((149.19572 -35.36126, 149.19970 -35.3...",12796,12743,13.7


In [29]:
poa_gdf.head()

,POA_CODE21,geometry
0,0800,"POLYGON ((130.83681 -12.45376, 130.83684 -12.4..."
1,0810,"POLYGON ((130.89986 -12.36567, 130.89875 -12.3..."
2,0812,"POLYGON ((130.91915 -12.40786, 130.91831 -12.4..."
3,0820,"POLYGON ((130.85260 -12.43994, 130.85089 -12.4..."
4,0822,"MULTIPOLYGON (((136.56648 -12.08392, 136.56704..."


In [30]:
sa2.iloc[0]['2020(no.)']

4273

In [31]:
def check_for_population(geom, sa2_gdf=sa2):
    total_area = geom.area
    sum_2020 = 0
    sum_2021 = 0
    for i in range(len(sa2)):
        if (total_area>0 and geom.intersects(sa2.iloc[i].geometry) == True):
            current_area = geom.intersection(sa2.iloc[i].geometry).area
            total_area -= current_area
            proportion = current_area / sa2.iloc[i].geometry.area
            sum_2020 += sa2.iloc[i]['2020(no.)']*proportion
            sum_2021 += sa2.iloc[i]['2021(no.)']*proportion
            # print(sum_2020, sum_2021, proportion, sa2.iloc[i]['2020(no.)'])
    return sum_2020, sum_2021


In [32]:
poa_gdf['2020(no.)'] = None
poa_gdf['2021(no.)'] = None
for i in range(len(poa_gdf)):
    if(i%25==0):
        print(i)
    temp = poa_gdf.iloc[i]
    sum_2020, sum_2021 = check_for_population(temp.geometry)
    poa_gdf.iloc[i,2]=sum_2020
    poa_gdf.iloc[i,3]=sum_2021

0
25
50
75
100
125
150
175
200
225
250
275
300
325
350
375


KeyboardInterrupt: 

In [ ]:
poa_gdf.head()

,POA_CODE21,2020(no.),2021(no.),geometry
0,800,7708.999833965965,7678.999835068511,"POLYGON ((130.83681 -12.45376, 130.83684 -12.4..."
1,810,36381.99921240709,36215.99921668285,"POLYGON ((130.89986 -12.36567, 130.89875 -12.3..."
2,812,19924.00036161399,19888.00036054116,"POLYGON ((130.91915 -12.40786, 130.91831 -12.4..."
3,820,21186.87673475262,21159.06665843644,"POLYGON ((130.85260 -12.43994, 130.85089 -12.4..."
4,822,35661.58644259583,35569.3627175903,"MULTIPOLYGON (((136.56648 -12.08392, 136.56704..."


In [ ]:
poa_gdf = poa_gdf.drop('geometry', axis=1)

In [ ]:
poa_gdf.to_parquet('../data/curated/geo_pos_population.parquet', index=False)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
  """Entry point for launching an IPython kernel.
